In [1]:
import numpy as np
import pandas as pd
import re

import nltk


In [177]:
def linear_regression(X, y):
    X = pd.get_dummies(X)._get_numeric_data().values
    y = y.values
    
    intercept_col = np.ones(X.shape[0]).reshape(X.shape[0],1)
    X = np.hstack((intercept_col, X))
    b = np.linalg.inv(X.T@X)@X.T@y
    return b

def rmse(slopes, X_test, y_test):
    X_test = pd.get_dummies(X_test)._get_numeric_data().values
    intercept_col = np.ones(X_test.shape[0]).reshape(X_test.shape[0],1)
    X_test = np.hstack((intercept_col, X_test))
    y_val_pred = X_test @ b
    y_val = y_test.values
    return np.sqrt(((y_val - y_val_pred) ** 2).mean())
    
def train_test_split(frac, X, response = 'Volume Sold (Gallons)'):
    X_train = X.sample(frac = frac)
    X_test = X.drop(X_train.index, axis = 0)
    
    
    GROUP_VARIABLES = ['County', 'Year','Month','Pack', 'Category Name']
    NUMERIC_VARIABLES = ['Bottle Volume (ml)','State Bottle Cost', 'State Bottle Retail',
                     'Bottles Sold','Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)',
                        'POPESTIMATE', 'BIRTHS', 'DEATHS', 'NATURALINC', 'INTERNATIONALMIG',
                         'DOMESTICMIG', 'NETMIG', 'RESIDUAL', 'NPOPCHG_', 'GQESTIMATES']
    
    train_pivot =  pd.pivot_table(X_train, 
                                index = GROUP_VARIABLES,
                                values = NUMERIC_VARIABLES,
                                aggfunc = np.mean).reset_index()
    X_train = train_pivot.drop(response, axis = 1)
    y_train = train_pivot[response]
    
    test_pivot = pd.pivot_table(X_test, 
                                index = GROUP_VARIABLES,
                                values = NUMERIC_VARIABLES,
                                aggfunc = np.mean).reset_index()
    X_test = test_pivot.drop(response, axis = 1)
    y_test = test_pivot[response]
    
    return X_train,X_test, y_train, y_test

def r2(slopes, X, y):
    X = pd.get_dummies(X)._get_numeric_data().values
    y = y.values
    intercept_col = np.ones(X.shape[0]).reshape(X.shape[0],1)
    X = np.hstack((intercept_col, X))
    
    ss_tot = ((y - y.mean()) ** 2).sum()
    
    y_val_pred = X @ b
    ss_res = ((y - y_val_pred) ** 2).sum()
    print(y,y_val_pred)
    return 1 - (ss_res/ss_tot)

In [3]:
MAIN_DATASET = "data/iowaliquor.csv"
#MAIN_DATASET = "data/iowa-sample.csv"

In [4]:
df = pd.read_csv(MAIN_DATASET)

/Users/anishyakkala/Library/Python/3.6/lib/python/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [62]:
df["County"] = df["County"].str.lower()

In [68]:
df_raw_county = pd.read_csv("data/co-est2019-alldata.csv", encoding = "ISO-8859-1")
df_raw_county["id"] = df_raw_county.index

In [69]:
df_county_year = pd.wide_to_long(df_raw_county.drop(["SUMLEV","REGION","DIVISION","STATE","COUNTY","CENSUS2010POP","ESTIMATESBASE2010"],axis = 1),
                  ["POPESTIMATE","NPOCHG_","BIRTHS",
                  "DEATHS","NATURALINC","INTERNATIONALMIG",
                  "DOMESTICMIG","NETMIG","RESIDUAL",
                  "GQESTIMATESBASE","RBIRTH","RDEATH","RNATURALINC",
                  "RINTERNATIONALMIG","RDOMESTICMIG","RNETMIG","NPOPCHG_","GQESTIMATES"], i="id", j="year")

In [70]:
df_county_year["CTYNAME"] = df_county_year["CTYNAME"].str.replace(" County", "").str.lower()

In [71]:
df_county_year = df_county_year[df_county_year["STNAME"] == "Iowa"]

In [72]:
df_county_year = df_county_year.drop("NPOCHG_", axis = 1).dropna(axis = 1)

In [49]:
df["Year"] = pd.DatetimeIndex(df['Date']).year
df["Month"] = pd.DatetimeIndex(df['Date']).month

In [94]:
df_full = df_pivot_table.merge(df_county_year, 'inner', left_on = ['Year','County'], right_on = ['year','CTYNAME']).dropna()

In [176]:
df_full.columns

Index(['County', 'Year', 'Month', 'Pack', 'Category Name',
       'Bottle Volume (ml)', 'Bottles Sold', 'Sale (Dollars)',
       'State Bottle Cost', 'State Bottle Retail', 'Volume Sold (Gallons)',
       'Volume Sold (Liters)', 'id', 'year', 'CTYNAME', 'STNAME',
       'POPESTIMATE', 'BIRTHS', 'DEATHS', 'NATURALINC', 'INTERNATIONALMIG',
       'DOMESTICMIG', 'NETMIG', 'RESIDUAL', 'NPOPCHG_', 'GQESTIMATES'],
      dtype='object')

In [178]:
X_train,X_test, y_train, y_test = train_test_split(.5, df_full)

In [179]:
b = linear_regression(X_train, y_train)

In [180]:
rmse(b, X_test, y_test)

52.76037917433177

In [182]:
X_train._get_numeric_data()

,Year,Month,Pack,BIRTHS,Bottle Volume (ml),Bottles Sold,DEATHS,DOMESTICMIG,GQESTIMATES,INTERNATIONALMIG,NATURALINC,NETMIG,NPOPCHG_,POPESTIMATE,RESIDUAL,Sale (Dollars),State Bottle Cost,State Bottle Retail,Volume Sold (Liters)
0,2012,1,6,80.0,1750.0,4.888889,104.0,-56.0,158.0,1.0,-24.0,-55.0,-78.0,7468.0,1.0,48.660000,6.396667,9.821111,8.555556
1,2012,1,6,80.0,1750.0,5.600000,104.0,-56.0,158.0,1.0,-24.0,-55.0,-78.0,7468.0,1.0,77.958000,9.100000,13.729000,9.800000
2,2012,1,6,80.0,1750.0,8.571429,104.0,-56.0,158.0,1.0,-24.0,-55.0,-78.0,7468.0,1.0,137.071429,10.754286,16.129524,15.000000
3,2012,1,6,80.0,750.0,6.000000,104.0,-56.0,158.0,1.0,-24.0,-55.0,-78.0,7468.0,1.0,162.780000,18.080000,27.130000,4.500000
4,2012,1,6,80.0,750.0,1.000000,104.0,-56.0,158.0,1.0,-24.0,-55.0,-78.0,7468.0,1.0,43.250000,28.830000,43.250000,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459029,2019,12,24,154.0,375.0,16.727273,161.0,-132.0,199.0,11.0,-7.0,-121.0,-128.0,12562.0,0.0,101.156364,4.229091,6.347273,6.272727
459030,2019,12,24,154.0,375.0,6.000000,161.0,-132.0,199.0,11.0,-7.0,-121.0,-128.0,12562.0,0.0,49.500000,5.500000,8.250000,2.250000
459031,2019,12,24,154.0,375.0,15.571429,161.0,-132.0,199.0,11.0,-7.0,-121.0,-128.0,12562.0,0.0,116.785714,5.000000,7.500000,5.838571
459032,2019,12,24,154.0,375.0,9.000000,161.0,-132.0,199.0,11.0,-7.0,-121.0,-128.0,12562.0,0.0,101.250000,7.500000,11.250000,3.372500


In [169]:
b

array([ 6.90698342e+13,  2.57168164e+01,  3.20149738e+00,  1.72217376e-01,
        3.82902683e-04, -1.62180304e-01,  1.66710998e-03, -3.73747633e+02,
        2.49146643e+02,  3.06000040e-01, -6.90698342e+13, -6.90698342e+13,
       -6.90698342e+13, -6.90698342e+13, -6.90698342e+13, -6.90698342e+13,
       -6.90698342e+13, -6.90698342e+13, -6.90698342e+13, -6.90698342e+13,
       -6.90698342e+13, -6.90698342e+13, -6.90698342e+13, -6.90698342e+13,
       -6.90698342e+13, -6.90698342e+13, -6.90698342e+13, -6.90698342e+13,
       -6.90698342e+13, -6.90698342e+13, -6.90698342e+13, -6.90698342e+13,
       -6.90698342e+13, -6.90698342e+13, -6.90698342e+13, -6.90698342e+13,
       -6.90698342e+13, -6.90698342e+13, -6.90698342e+13, -6.90698342e+13,
       -6.90698342e+13, -6.90698342e+13, -6.90698342e+13, -6.90698342e+13,
       -6.90698342e+13, -6.90698342e+13, -6.90698342e+13, -6.90698342e+13,
       -6.90698342e+13, -6.90698342e+13, -6.90698342e+13, -6.90698342e+13,
       -6.90698342e+13, -